#PROLOG CODIGO

In [ ]:
% ================================
%  SISTEMA DE PEDIDOS - RESTAURANTE
%  (reglas deterministas)
% ================================

% ---- Hechos de clientes (ejemplo) ----
pedido(ana, hamburguesa).
pedido(ana, papas).
pedido(ana, refresco).

pedido(luis, ensalada).
pedido(luis, agua).

pedido(carla, pizza).
pedido(carla, alitas).
pedido(carla, cerveza).

pedido(mario, hamburguesa).
pedido(mario, malteada).

edad(ana, 25).
edad(luis, 70).
edad(carla, 30).
edad(mario, 15).

% ---- 1) ALERTAS (restricciones fuertes) ----
% Clientes menores de 18 no pueden consumir alcohol
alerta_producto(cerveza).
alerta_producto(vino).

tiene(C, P) :- pedido(C, P).

detectar_alerta(C, P) :-
    alerta_producto(P),
    tiene(C, P),
    edad(C, E), E < 18, !.   % corte: si hay alerta, se termina

% ---- 2) VULNERABILIDAD ----
% Clientes mayores de 65 años o con dieta especial
vulnerable(C) :- edad(C, E), E >= 65.
vulnerable(luis).   % Luis tiene dieta especial

% ---- 3) MENÚ Y CLASIFICACIÓN ----
combo(clasico, [hamburguesa, papas, refresco]).
combo(familiar, [pizza, alitas, refresco]).
combo(saludable, [ensalada, agua]).
combo(postre, [malteada, papas]).

% Contar coincidencias entre pedido y combos
puntaje(C, Combo, N) :-
    combo(Combo, Lista),
    findall(P, (member(P, Lista), tiene(C, P)), A), length(A, N).

mejor_combo(C, Combo) :-
    findall(N, puntaje(C, _, N), Ns),
    Ns \= [], max_list(Ns, Max),
    puntaje(C, Combo, Max).

% ---- 4) PLAN ----
plan_para(alerta(_), revisar_pedido).
plan_para(combo(saludable), descuento_salud).
plan_para(combo(clasico), promo_refill).
plan_para(combo(familiar), combo_descuento).
plan_para(combo(postre), dos_por_uno).

ajustar_por_vulnerable(Plan, true, control_nutricional(Plan)).
ajustar_por_vulnerable(Plan, false, Plan).

% ---- 5) ORQUESTADOR ----
evaluar_cliente(C, reporte{
    riesgo: Riesgo,
    combos: Combos,
    plan: PlanAjustado,
    notas: Notas
}) :-
    (   detectar_alerta(C, P) ->
        Riesgo = alerta,
        Combos = [],
        plan_para(alerta(P), Plan),
        ajustar_por_vulnerable(Plan, false, PlanAjustado),
        Notas = [alerta_producto(P), "retirar del pedido"]
    ;   Riesgo = normal,
        (vulnerable(C) -> Vul = true ; Vul = false),
        findall(Co, mejor_combo(C, Co), Combos),
        decidir_plan(Combos, Vul, PlanAjustado, Notas)
    ).

decidir_plan([Co], Vul, PlanAjustado, [combo_unico(Co)]) :-
    plan_para(combo(Co), Plan),
    ajustar_por_vulnerable(Plan, Vul, PlanAjustado), !.

decidir_plan([Co1, Co2 | _], Vul, PlanAjustado, [combo_ambiguo([Co1, Co2])]) :-
    % si hay empate, se da un beneficio genérico
    Base = regalo_extra,
    ajustar_por_vulnerable(Base, Vul, PlanAjustado), !.

decidir_plan([], _, sin_datos, ["no coincide con combos conocidos"]).


### Explicación de mi programa en prolog

1. Hechos de clientes

pedido(ana, hamburguesa).

pedido(ana, papas).

pedido(ana, refresco).

Significa que Ana pidió una hamburguesa, papas y refresco.
También se guarda la edad de cada cliente para después tomar decisiones

2. Alertas

alerta_producto(cerveza).

Algunas cosas están prohibidas.

y la regla:

detectar_alerta(C, P) :-

alerta_producto(P),

tiene(C, P),

 edad(C, E), E < 18, !.

 Quiere decir: si un menor pide cerveza o vino → se activa una alerta y se corta el flujo.
En la vida real sería como que el sistema dice: “No se puede vender alcohol a este cliente”.

3. Vulnerabilidad

Se considera vulnerable:

* Una persona mayor de 65 años.

* O alguien con dieta especial.

Esto afecta el plan final (ejemplo: darles un control nutricional extra).

4. Menú y combos

Aquí se definen los combos:

combo(clasico, [hamburguesa, papas, refresco]).

combo(familiar, [pizza, alitas, refresco]).

combo(saludable, [ensalada, agua]).

El sistema compara lo que pidió el cliente con los combos para ver con cuál coincide más.
Ejemplo: si alguien pidió hamburguesa, papas y refresco, seguramente está pidiendo el combo clásico.

5. Planes

Dependiendo del combo o situación, se asigna un beneficio:

* Combo clásico → promo de refill.

* Combo saludable → descuento salud.

* Si hay alerta → revisar el pedido.

Además, si el cliente es vulnerable, se ajusta con un plan especial (ejemplo: control nutricional).

6. Orquestador

La regla principal es:

evaluar_cliente(C, reporte{...})

Aquí se junta todo:

1. Si hay alerta → se bloquea el pedido.

2. Si no, se revisa la vulnerabilidad y los combos.

3. Se decide el plan final (promoción, descuento o advertencia).

4. Se genera un reporte con:

* Riesgo (alerta o normal)

* Combo detectado

* Plan (qué hacer)

* Notas adicionales

En pocas palabras: este código funciona como un cajero inteligente de restaurante que no solo registra lo que pide el cliente, sino que también detecta promociones, prohíbe cosas indebidas (como alcohol a menores) y ajusta beneficios según la edad o condiciones.